In [46]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
import ast
import re
stop_words = set(stopwords.words('english'))

In [67]:
files = glob.glob('/Users/jane/Desktop/search+youtube_v4/*.csv')
df_news = df_science = df_jobs = df_business = pd.DataFrame(columns=pd.read_csv(files[0]).columns)
for f in files: 
    df = pd.read_csv(f)
    df_news = df_news.append(df[df['all_categories'] == "News"], ignore_index=True)
    df_science = df_science.append(df[df['all_categories'] == "Science"], ignore_index=True)
    df_jobs = df_jobs.append(df[df['all_categories'] == "Jobs & Education"], ignore_index=True)
    df_business = df_business.append(df[df['all_categories'] == "Business & Industrial"], ignore_index=True)
df_business


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,ratings,action,dislikes,likes,...,days,hours,minutes,weekdays,link,titles,source,query,datetime,all_categories
0,53,53,53,53,823,823,NaN,Searched,NaN,NaN,...,7,2,37,1,NaN,NaN,search,google's income per year,2020-01-07 02:37:00,Business & Industrial
1,55,55,55,55,821,821,NaN,Searched,NaN,NaN,...,7,2,37,1,NaN,NaN,search,google's profit per year,2020-01-07 02:37:00,Business & Industrial
2,57,57,57,57,819,819,NaN,Searched,NaN,NaN,...,7,2,40,1,NaN,NaN,search,mtn's income yearly,2020-01-07 02:40:00,Business & Industrial
3,60,60,60,60,816,816,NaN,Searched,NaN,NaN,...,7,2,45,1,NaN,NaN,search,google ceo salary 2019,2020-01-07 02:45:00,Business & Industrial
4,105,105,105,105,771,771,NaN,Searched,NaN,NaN,...,7,19,44,1,NaN,NaN,search,huntsman program acceptance rate,2020-01-07 19:44:00,Business & Industrial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,4088,4088,4088,4088,802,459,NaN,Searched,NaN,NaN,...,1,21,56,4,NaN,NaN,search,delta pressure washer hose replacement,2020-05-01 21:56:00,Business & Industrial
5057,4130,4130,4130,4130,763,420,NaN,Searched,NaN,NaN,...,4,17,43,0,NaN,NaN,search,limit chain rule,2020-05-04 17:43:00,Business & Industrial
5058,4277,4277,4277,4277,624,281,NaN,NaN,NaN,NaN,...,14,18,58,3,NaN,NaN,search,trolley,2020-05-14 18:58:00,Business & Industrial
5059,4479,4479,4479,4479,430,87,NaN,Searched,NaN,NaN,...,21,19,1,3,NaN,NaN,search,order the events that occur in one cycle of th...,2020-05-21 19:01:00,Business & Industrial


In [78]:
def query_list(df_name, month_number):
    df_month = df_name[df_name['months'] == month_number]
    searches = df_month['query'].tolist()
    tokenized = []
    punct_list = '[,.:@#?!&$=-()]'
    
    for q in searches:
        tokenlist = word_tokenize(q)
        for token in tokenlist:
            tokenized.append(token)
    tokenized = list(set(tokenized).difference(stop_words))
    tokenized = list(set(tokenized).difference(punct_list))
    return tokenized

query_list(df_business, 2)

        

['cons',
 'Effective',
 'Commons',
 'myMailHouse',
 'glass',
 'proprietary',
 'identifying',
 'Development',
 'aon',
 'solution',
 'Aerospace',
 'Data',
 'photographer',
 'Locations',
 'consulting',
 'Potassium',
 'bezos',
 'Fairs',
 'desk',
 'IndieBio',
 'implementing',
 'tape',
 'Conversions',
 'Why',
 'Employment',
 '1040ez',
 'place',
 'raytheon',
 'Generator',
 'Collin',
 'Cooling',
 'good',
 'Authority',
 'USD',
 'Ad',
 'Proctor',
 'Every',
 'WiFi',
 'estimated',
 'Its',
 'College',
 'Indiana',
 'antifreeze',
 'NRDC',
 'Fraser',
 'bullet',
 'Principal',
 'brand',
 'sequencing',
 'Lab',
 'worked',
 'F2480',
 'Facebook',
 'sql',
 'Pharma',
 'Network',
 '+',
 'edX',
 'fabric',
 'droplet',
 'jeff',
 'plastic',
 'entry',
 'postmaster',
 'diagnostics',
 'simon',
 'peru',
 'Invensis',
 'Cool',
 'Indotronix',
 'LLC',
 '9',
 'farms',
 'uphill',
 'Missouri',
 'effectiveness',
 'tree',
 'Mail',
 'Rolling',
 'Service',
 'Donald',
 'Financial',
 '2',
 'promotion',
 'orange',
 'plaster',
 'FIN

In [80]:
query_list(df_news, 1)

['Keene',
 '//qz.com/quartzy/1262785/best-time-to-exercise-the-pros-and-cons-of-morning-and-evening-workouts/amp/',
 '``',
 'Your',
 'strike',
 '//www.timesunion.com/news/amp/Once-lively-Saratoga-County-sanatorium-a-grim-13937126.php',
 '//m.huffpost.com/us/entry/12376168/amp',
 'Snow',
 'phl',
 'york',
 'LA',
 '–',
 'less',
 'Aggressively',
 'sanders',
 'pledge',
 'Lizzo',
 'Mountains',
 'spiritual',
 'wife',
 'know',
 'Cardi',
 'Harry',
 'B',
 'estimates',
 'vote',
 'democratic',
 '2019',
 'Cohen',
 'Why',
 'castle',
 'FiveThirtyEight',
 'JetBlue',
 'Her',
 'departamento',
 '//amp.cnn.com/cnn/videos/politics/2020/01/10/stephanie-grisham-again-ridiculist-sot-ac360-vpx.cnn',
 'Presidential',
 'linke',
 'Day',
 'weather',
 'rothschild',
 'mill',
 'CNN.com',
 'Markle',
 'DPC',
 'next',
 'People',
 'miley',
 'vs.',
 'paid',
 'Parker',
 '//amp.theguardian.com/music/2020/jan/18/mac-miller-circles-review',
 'josep',
 '//www.washingtonpost.com/opinions/2020/01/22/sos-please-help-me-worlds-gre

In [71]:
query_list(df_jobs, 2)

['functions',
 'Starbucks',
 'So',
 '``',
 'Commons',
 'richards',
 '7:1',
 'SYEP',
 '3.72',
 'Hour',
 'Databases',
 'Liberal',
 'libraries',
 'Development',
 'adolescents',
 'solution',
 'Appreciation',
 'deans',
 'Data',
 'digital',
 'AstraZeneca',
 'Locations',
 'infectious',
 'WTAMU',
 'teaching',
 'Visine',
 'B',
 'Fairs',
 '009B',
 'Shoulder',
 'Because',
 'memory',
 'toefl',
 'sudoku',
 'Consultant',
 'asu',
 'Studenthood',
 'Employment',
 'Requests',
 '0',
 'thornton',
 'library',
 'fraud',
 'randal',
 'Integrative',
 'P3-51',
 'place',
 'calendars',
 'Declaring',
 'Leap',
 'security',
 'Coefficients',
 'D',
 'concept',
 'Are',
 'Improving',
 'fraternity',
 'tutor',
 'Octopus',
 'corelogic',
 'Practical',
 'Nelson',
 'LiquidPlanner',
 'tie',
 'awarded',
 'quizzes',
 'Eastman',
 'Detailed',
 'Forces',
 'Length',
 'anderson',
 'czaplicki',
 'Down',
 'jec',
 'Topic',
 'supporting',
 'Emergency',
 'ESL',
 'Pandemic',
 'Textbooks',
 'phasor',
 'CSUN.edu',
 'College',
 'alexander',
 

In [79]:
query_list(df_science, 3)

['Neuron',
 'functions',
 'Concave',
 'listener',
 '``',
 'API',
 'RoseIndia',
 'lisp',
 'PACF/ACF',
 'GitHub',
 'Aerospace',
 'Data',
 'digital',
 'expression',
 'Python',
 'B',
 'butterfly',
 'precentral',
 'Kernel',
 'LAN',
 'view',
 'Fish',
 'tests',
 'Spectroscopy',
 'React',
 'Why',
 'imarkkeyz',
 'italy',
 'notebook',
 'voltage',
 'end',
 'Inference',
 'library',
 'Sciencing',
 '0',
 'place',
 'pi/3',
 'Understanding',
 'force',
 '=Bsinθ+k',
 'Anaconda',
 'Are',
 'postcentral',
 'Pieces',
 'Proving',
 'js',
 '0,3π',
 'es',
 'expressions',
 'Differences',
 'good',
 'computational',
 'Ceiling',
 'ouverte',
 'Electrical',
 'evaporation',
 'bayesian',
 'Every',
 'AR',
 'Revolution',
 'insertion',
 'Khan',
 'College',
 'supervised',
 'Doctor',
 'wound',
 'make',
 'Polar',
 'exact',
 '2.5',
 'rectum',
 'Card',
 'Interpret',
 'sql',
 'Network',
 '+',
 'Eukaryotes',
 'x-axis',
 'algorithm',
 'worksheet',
 'Small',
 'Expected',
 'scipy',
 'Sound',
 'notes',
 'gives',
 'Inertia',
 'SPSS',